### 데이터셋 준비 하기
Instruction Fine Tuning을 위해 데이터셋을 준비합니다.  
데이터셋은 아래의 순서로 준비합니다.
---

1. 도메인 정하기  
부동산에 대해 잘 대답할 수 있는 챗봇을 타겟으로 설정합니다.  

2. 부동산에 관련된 키워드를 생각해보기  
책, 뉴스, 부동산 사이트에서 키워드를 수집합니다. 사람들이 궁금해할만한 키워드를 대상으로 합니다. (예: 전세 계약, 신혼부부 특별공급, 토지거래허가구역...)  

3. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트 만들어보기(`query.jsonl`)  
2번에서 수집한 키워드를 기반으로, 사람들이 궁금해할만한 질문리스트를 ChatGPT를 활용해 만들어봅니다. 충분한 데이터셋 확보할 만큼의 질문리스트를 생성합니다.

4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기(`search_data.json`)  
3번에서 수집한 질문리스트를 Selenium 라이브러리를 활용하여 네이버에 검색합니다. 검색 결과중 인기글의 텍스트 데이터를 추출합니다.

5. encoder 모델을 활용하여 데이터를 유사도 기준으로 정렬하기(`search_data.json`)  
4번에서 수집한 인기글 텍스트 데이터가 질문리스트와 얼마나 유사한지 계산합니다.
질문리스트와 가장 유사한 인기글 텍스트를 상위 순위로 정렬합니다.

6. Instuction 데이터셋 만들기(`instruction.jsonl`)  
질문리스트 + 정렬한 인기글을 합쳐서 ChatGPT에 Instruction 데이터를 만들어달라고 요청합니다. 이 Instruction 데이터는 Fine Tuning에 사용됩니다.

In [2]:
!pip install --quiet\
selenium==4.20.0\
openai==1.23.6\
datasets==2.19.0\
accelerate==0.27.2\
flash-attn==0.2.4\
peft==0.10.0\
trl==0.8.6\
transformers==4.40.1\
python-dotenv==1.0.1

In [1]:
import json
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

import utils
import prompts
import similarity

### 2. 부동산에 관련된 키워드를 생각해보기
부동산에 관련된 키워드를 수집하여 저장합니다.  
예시 데이터는 `seed_words.txt` 파일로 제공합니다.

In [12]:
with open("./data/seed_words.txt", "r") as f:
    seed_words = f.readlines()

In [13]:
seed_words[:4]

['전세 계약\n', '임대차 계약\n', '전세 사기\n', '임대차 분쟁\n']

In [14]:
# Element마다 있는 \n 제거, '전세 계약\n' -> '전세 계약'
seed_words = list(map(lambda x: x.strip("\n"), seed_words))

In [15]:
seed_words[:4]

['전세 계약', '임대차 계약', '전세 사기', '임대차 분쟁']

### 3. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트 만들어보기
`format` 메서드를 적용할 수 있게 프롬프트를 미리 작성합니다.  
미리 작성된 프롬프트에 `seed_words`를 `format` 적용하여 프롬프트를 완성시킵니다.  
예시 데이터는 `query.jsonl` 파일로 제공합니다.

In [16]:
print(prompts.SEED_WORD_PROMPT_PREFIX + prompts.SEED_WORD_PROMPT_CONTENT)

주어진 seed_word에 대해 궁금해할 질문을 10개를 생성하세요.
만들어낸 질문은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
아래 양식으로 출력하세요:
{{"seed_word": "{seed_word}", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}}


In [17]:
print(prompts.SEED_WORD_PROMPT_PREFIX + prompts.SEED_WORD_PROMPT_CONTENT.format(seed_word=seed_words[0]))

주어진 seed_word에 대해 궁금해할 질문을 10개를 생성하세요.
만들어낸 질문은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
아래 양식으로 출력하세요:
{"seed_word": "전세 계약", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}


In [18]:
# 모델에게 Prefix 토큰을 중복해서 전달할 필요가 없으므로, Prefix 토큰은 1번만 사용하고 Content을 반복해서 프롬프트를 생성
seed_word_prefix = prompts.SEED_WORD_PROMPT_PREFIX
seed_word_content = [prompts.SEED_WORD_PROMPT_CONTENT.format(seed_word=s) for s in seed_words]

In [19]:
# 모델에게 Prefix 토큰을 중복해서 전달할 필요가 없으므로, Prefix 토큰은 1번만 사용하고 Content을 반복해서 프롬프트를 생성
# 1개의 Prefix 마다 10개의 Content를 추가한다
seed_word_prompts = []
total_prompts = len(seed_words) // 10
for idx in range(1, total_prompts+2):
    start_index = (idx -1) * 10
    end_index = idx * 10
    seed_word_prompt = seed_word_prefix + "\n".join(seed_word_content[start_index:end_index])
    seed_word_prompts.append(seed_word_prompt)

In [20]:
print(seed_word_prompts[9])

주어진 seed_word에 대해 궁금해할 질문을 10개를 생성하세요.
만들어낸 질문은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
아래 양식으로 출력하세요:
{"seed_word": "장기보유 특별 공제", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "주택임대사업자", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "종부세 중과세", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "보금자리론", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "디딤돌대출", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "정책모기지", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}


### 4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기
3번에서 생성한 질문리스트를 selenium 라이브러리를 통해 네이버로 검색합니다.  
검색 결과의 인기글의 텍스트 정보를 저장합니다.  
예시 데이터는 `search.jsonl`로 제공합니다.  

In [21]:
query_data = utils.jload("./data/query.jsonl")

In [22]:
query_data[:2]

['{"seed_word": "전세 계약", "answer": ["전세 계약 기간은 보통 얼마나 되나요?", "전세 계약서에는 어떤 내용이 포함되어야 하나요?", "전세 계약 시 부동산 중개수수료는 어떻게 책정되나요?", "전세 계약 갱신 시 주의해야 할 점은 무엇인가요?", "전세 계약 종료 시 보증금 반환은 어떻게 이루어지나요?", "전세 계약 중 집주인이 바뀌면 어떻게 해야 하나요?", "전세 계약 시 등기부등본은 왜 확인해야 하나요?", "전세 계약 기간 중 월세로 전환하는 것이 가능한가요?", "전세 계약 시 확인해야 할 집의 하자 사항은 무엇인가요?", "전세 계약 분쟁 발생 시 어떤 법적 대응을 할 수 있나요?"]}\n',
 '{"seed_word": "임대차 계약", "answer": ["임대차 계약서에는 어떤 내용이 포함되어야 하나요?", "임대차 계약 기간은 일반적으로 얼마나 되나요?", "임대차 계약 시 임차인이 부담해야 하는 비용은 무엇인가요?", "임대차 계약 종료 시 임차인의 원상복구 의무는 어떻게 되나요?", "임대차 계약 기간 중 임대인이 집을 매도하면 어떻게 되나요?", "임대차 계약 갱신 거절 시 임대인이 내세울 수 있는 사유는 무엇인가요?", "임대차 계약 시 확인해야 할 특약 사항은 무엇인가요?", "임대차 계약 기간 중 임차인이 계약을 해지하고 싶다면 어떻게 해야 하나요?", "임대차 계약상 임차인의 권리와 의무는 무엇인가요?", "임대차 계약 분쟁 발생 시 어떤 법적 대응을 할 수 있나요?"]}\n']

In [23]:
queries = []
for line in query_data:
    query = json.loads(line)
    queries = queries + query['answer']

In [24]:
queries[:5]

['전세 계약 기간은 보통 얼마나 되나요?',
 '전세 계약서에는 어떤 내용이 포함되어야 하나요?',
 '전세 계약 시 부동산 중개수수료는 어떻게 책정되나요?',
 '전세 계약 갱신 시 주의해야 할 점은 무엇인가요?',
 '전세 계약 종료 시 보증금 반환은 어떻게 이루어지나요?']

In [25]:
len(queries)

960

selenium이 4.1 버전으로 업그레이드 되면서 별도로 webdriver를 다운로드 받을 필요가 없어졌습니다.  
webdriver 파일이 없으면 selenium이 자체적으로 다운로드하게 된다고 합니다. [stackoverflow](https://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-seleniumhttps://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-selenium)  
본 예제의 selenium 코드 크롤링 코드는 [wikidocs](https://wikidocs.net/137914) 내용을 참조하였습니다.  

In [26]:
search_data = utils.get_document_through_selenium(
    inputs=queries,
    n_documents=5,
    indent=4,
    save_path = "./data/document.json"
)

100%|███████████████████████████████████████████████████████████████████████████████| 960/960 [2:47:07<00:00, 10.45s/it]


In [31]:
print(utils.jload("./data/document.json")[0]['document'][0])

말해야 되나요? 주택임대차보호법 제 6조 제 1항 (계약의 갱신) 계약이 해지되기 6개월 전부터 2개월 전까지, 계약을 해지하겠다는 통보를 해야 한다. 즉, 법률상 늦어도 계약이 종료되기 2개월 전까지는 임대인에게 말해야 된다는 뜻인데요. 만약 해당 기간 안에 말하지 않았다면? 자동으로 연장되는 묵시적 갱신이 될 수 있습니다. 때문에, 종료 시점에 맞춰서 자금을...


가끔 크롤링이 안되는 경우가 있어서 한번 검수해줍니다.

In [2]:
search_data = utils.jload('./data/document.json')
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

Index [18] 번째 데이터가 수집되지 않았네요.
Index [298] 번째 데이터가 수집되지 않았네요.
Index [503] 번째 데이터가 수집되지 않았네요.
Index [507] 번째 데이터가 수집되지 않았네요.
Index [594] 번째 데이터가 수집되지 않았네요.
Index [693] 번째 데이터가 수집되지 않았네요.
Index [783] 번째 데이터가 수집되지 않았네요.
Index [933] 번째 데이터가 수집되지 않았네요.
Index [937] 번째 데이터가 수집되지 않았네요.


수집되지 않은 데이터를 한번더 크롤링하게 하고 `document.json`에 저장합니다.

In [3]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        search_data[idx] = utils.get_document_through_selenium(
            inputs=search_data[idx]['question'],
            n_documents=5,
            indent=4
        )[0]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.56s/it]


제대로 수집되었는지 또 검사해줍니다.

In [53]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

Index [693] 번째 데이터가 수집되지 않았네요.


원인 파악을 위해 해당 페이지로 검색해보면 인기글이 없으므로 데이터에서 삭제합니다.

In [4]:
# 삭제할 인덱스 모으기
remove_idx_list = []
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        remove_idx_list.append(idx)

# 삭제할 인덱스를 pop 
for idx in remove_idx_list:
    search_data.pop(idx)

In [8]:
utils.jsave(data=search_data, file="./data/document.json", mode="w", indent=4)

In [6]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

### 5. encoder 모델을 활용하여 데이터를 유사도 기준으로 정렬하기
최근에 공개된 intfloat의 e5 Multi Lingual 모델을 사용하여 유사도를 계산합니다.  
질문을 했을 때, 검색되는 인기글 데이터들중 유사도가 높은 순서대로 문서를 다시 정렬합니다.  
정렬된 순서대로 데이터를 좀더 많이 참조하도록 프롬프트를 통해 지시합니다.

In [9]:
search_data = utils.jload("./data/document.json")

e5모델의 자세한 내용은 [hugginface](https://huggingface.co/intfloat/e5-base-v2)를 참조하세요

In [10]:
for question in tqdm(search_data):
    question['document'], question['scores'] = similarity.sort_by_similarity(question['question'], question['document'])

  1%|▊                                                                                 | 10/959 [00:13<22:04,  1.40s/it]


KeyboardInterrupt: 

In [ ]:
utils.jsave(search_data, "./data/search_data.json")

### 6. Instuction 데이터셋 만들기
완성된 질문리스트와 인기글 데이터를 통해 ChatGPT에 정답을 출력하도록 요청합니다.  
ChatGPT는 아래의 프롬프트처럼 question에 대한 answer 답변을 출력합니다.  
완성된 question, answer 텍스트는 학습시킬 모델의 훈련용 데이터셋으로 전달됩니다.  
본 문서의 프롬프트는 [Stanford Alpaca의 프롬프트](https://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txthttps://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txt)를 참조하였습니다.  
예시 데이터는 `instruction.jsonl`로 제공합니다.

In [ ]:
print(prompts.INSTRUCTION_PROMPT_PREFIX + prompts.INSTRUCTION_PROMPT_CONTENT)

In [ ]:
search = utils.jload("./data/search_data.json")

In [ ]:
# 리스트 형태인 document 데이터를 하나로 합치기
for data in search:
    data['document'] = "\n".join([f"{idx+1}. {d}" for idx, d in enumerate(data['document'])])

In [ ]:
print(search[0]['document'])

In [ ]:
print(prompts.INSTRUCTION_PROMPT_CONTENT)

In [ ]:
prefix = prompts.INSTRUCTION_PROMPT_PREFIX
instructions = []
total_instructions = len(search) // 10
for index in range(1, total_instructions + 2):
    start_index = (index - 1) * 10
    end_index = index * 10
    content = "\n".join([
        prompts.INSTRUCTION_PROMPT_CONTENT.format(question = data['question'], document = data['document'])
        for data in search[start_index:end_index]
    ])
    instruction = prefix + content
    instructions.append(instruction)

In [ ]:
print(instructions[0])

포맷팅하여 완성한 프롬프트는 아래와 같습니다.  
아래 데이터를 ChatGPT API로 넘겨줍니다.

In [ ]:
print(instructions[0][:2000])

`gpt-4-turbo`는 `gpt-4` 보다 성능이 좋으면서 가격은 1/3입니다.  
`gpt-4-turbo`를 사용하길 추천드립니다.  
[OpenAI 가격 정책](https://openai.com/pricing)을 참조하세요

In [14]:
for inst in tqdm(instructions):
    result = utils.get_completion(inst, model="gpt-4-turbo-2024-04-09")
    with open("instruction.jsonl", "a", encoding="utf-8") as f:
        for line in result.split("\n"):
            f.write(line)
            f.write("\n")